In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

In [2]:
topic_file = '/Users/bew/Downloads/DataForThai_jupyter/DataForThai_Pandas&SubTopic/SubTopic_dataforthai.csv'
topic_table = pd.read_csv(topic_file)


topic_table['รหัส'] = topic_table['รหัส'].astype(str)
# Function to add '0' to strings of length 4
def add_zero(s):
    if len(s) == 4:
        return '0' + s
    else:
        return s
# Applying the function to the DataFrame
topic_table['รหัส'] = topic_table['รหัส'].apply(lambda x: add_zero(x))


# topic_table.dtypes
topic_table = topic_table[:62]
# topic_table = topic_table[608:628]
# topic_table = topic_table[topic_table['รหัส']=='46311']
# print(topic_table)
# max(topic_table['จำนวนหน้า'])
topic_table

,รหัส,ประเภทธุรกิจ,จำนวน,จำนวนหน้า
0,01111,การปลูกข้าวโพดที่ใช้เมล็ดแก่,24,1
1,01112,การปลูกธัญพืช (ยกเว้นข้าวและข้าวโพด),34,1
2,01113,การปลูกพืชตระกูลถั่ว,15,1
3,01114,การปลูกถั่วเหลือง,3,1
4,01115,การปลูกพืชน้ำมัน (ยกเว้นถั่วเหลือง),10,1
...,...,...,...,...
57,01461,การเลี้ยงไก่ไข่,244,1
58,01462,การเลี้ยงไก่เนื้อ,536,2
59,01463,การเลี้ยงเป็ด,22,1
60,01469,การเลี้ยงสัตว์ปีกอื่นๆ ซึ่งมิได้จัดประเภทไว้ใน...,173,1


In [3]:
len(topic_table)

62

In [4]:
# รวมไฟล์

# Initialize the result DataFrame
result = pd.DataFrame()

# Iterate through each row in topic_table
for index, row in topic_table.iterrows():
    
    value1 = row['รหัส']
    
    for i in range(1,150):
        i_str = str(i)  # Convert i to string
        file = f'/Users/bew/Downloads/DataForThai_jupyter/DataForThai_Pandas&SubTopic/pf2/{value1}_{i_str}.html'

        # Read the HTML file
        try:
            # Read the HTML file
            with open(file, 'r', encoding='utf-8') as file:
                html_content = file.read()
        except FileNotFoundError:
#             check file 1 ว่ามี 400 rows ไหม เผื่อพลาดข้อมูลหน้า 2+
            if i == 2:
                check_file_400 = f'/Users/bew/Downloads/DataForThai_jupyter/DataForThai_Pandas&SubTopic/pf2/{value1}_1.html'
                check_df_file_1 = pd.read_html(check_file_400)
                check_df_file_1 = check_df_file_1[1]
                if len(check_df_file_1) == 400:
                    print(f"{value1} have full 1 page (check for maybe have more pages)")
            # If the file is not found, break the loop
            break

        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'lxml')

        # Find the table (modify as needed to target the correct table)
        table = soup.find('table', {'class': 'datatable'})

        # Initialize an empty list to store the rows
        data = []

        # Iterate over each row in the table
        for row in table.find_all('tr', class_='click-able-row'):
            # Extract the text of each cell
            cells = row.find_all('td')
            row_data = [cell.text.strip() for cell in cells]

            # Extract the onclick attribute
            onclick = row.get('onclick')
            if onclick:
                # Extract the desired values from the onclick string
                parts = onclick.split("'")
                if len(parts) > 1:
                    company_id = parts[1]
                    company_name = parts[3]
                    row_data.append(company_id)
                    row_data.append(company_name)

            data.append(row_data)

        # Convert to DataFrame
        columns = ['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company ID', 'Company Name']
        df = pd.DataFrame(data, columns=columns)
        
        df['Capital'] = df['Capital'].str.replace(',', '')
        df['Capital'] = pd.to_numeric(df['Capital'])
        df = df[(df['Capital'] >= 30000000)]
        df = df.reset_index(drop=True)
        
        df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company Name'])

        # Concatenate df and result DataFrames
        result = pd.concat([result, df], axis=0, ignore_index=True)
        
        
#     # Specify the file name and file path
#     file_name = value1+'.csv'
#     file_path = '/Users/bew/Downloads/WebScrapperFor_DataForThai/DataForThaiCompanyIdsByCategories/'

#     # Combine the file path and file name
#     full_path = os.path.join(file_path, file_name)

#     result.to_csv(full_path, sep=',', index=False, header=True, encoding='utf-8')
    print(f"{value1} Complete")
#     df_null = []
#     result = pd.DataFrame(df_null, columns=columns)
#     result = result.drop(columns=['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company Name'])

    # Reset the index of the final result DataFrame
#     result = result.reset_index(drop=True)

# result

01111 Complete
01112 Complete
01113 Complete
01114 Complete
01115 Complete
01121 Complete
01122 Complete
01131 Complete
01132 Complete
01133 Complete
01135 Complete
01136 Complete
01139 Complete
01140 Complete
01150 Complete
01161 Complete
01169 Complete
01191 Complete
01192 Complete
01193 Complete
01194 Complete
01199 Complete
01210 Complete
01221 Complete
01222 Complete
01223 Complete
01224 Complete
01225 Complete
01226 Complete
01227 Complete
01229 Complete
01231 Complete
01239 Complete
01249 Complete
01251 Complete
01259 Complete
01261 Complete
01262 Complete
01269 Complete
01271 Complete
01272 Complete
01279 Complete
01281 Complete
01282 Complete
01289 Complete
01291 Complete
01292 Complete
01299 Complete
01301 Complete
01302 Complete
01411 Complete
01412 Complete
01419 Complete
01420 Complete
01441 Complete
01442 Complete
01450 Complete
01461 Complete
01462 Complete
01463 Complete
01469 Complete
01492 Complete


In [5]:
result = result.reset_index(drop=True)

result

,Company ID
0,0505555004367
1,0515558000759
2,0105526007273
3,0105536067431
4,0105533070679
...,...
552,0255560002362
553,0725544000555
554,0305569005230
555,0305562006712


In [6]:
len(result)                                                     # ข้อมูลบริษัททั้งหมด = 44,318 (แต่ยังไม่กรองการดำเนินธุรกิจ)

557

In [7]:
# # แยกไฟล์ตามหัวข้อ

# # Initialize the result DataFrame
# result = pd.DataFrame()

# # Iterate through each row in topic_table
# for index, row in topic_table.iterrows():
    
#     value1 = row['รหัส']
    
#     for i in range(1,150):
#         i_str = str(i)  # Convert i to string
#         file = f'/Users/bew/Downloads/DataForThai_Pandas&SubTopic/pf2/{value1}_{i_str}.html'

#         # Read the HTML file
#         try:
#             # Read the HTML file
#             with open(file, 'r', encoding='utf-8') as file:
#                 html_content = file.read()
#         except FileNotFoundError:
# #             check file 1 ว่ามี 400 rows ไหม เผื่อพลาดข้อมูลหน้า 2+
#             if i == 2:
#                 check_file_400 = f'/Users/bew/Downloads/DataForThai_Pandas&SubTopic/pf2/{value1}_1.html'
#                 check_df_file_1 = pd.read_html(check_file_400)
#                 check_df_file_1 = check_df_file_1[1]
#                 if len(check_df_file_1) == 400:
#                     print(f"{value1} have full 1 page (check for maybe have more pages)")
#             # If the file is not found, break the loop
#             break

#         # Parse the HTML content
#         soup = BeautifulSoup(html_content, 'lxml')

#         # Find the table (modify as needed to target the correct table)
#         table = soup.find('table', {'class': 'datatable'})

#         # Initialize an empty list to store the rows
#         data = []

#         # Iterate over each row in the table
#         for row in table.find_all('tr', class_='click-able-row'):
#             # Extract the text of each cell
#             cells = row.find_all('td')
#             row_data = [cell.text.strip() for cell in cells]

#             # Extract the onclick attribute
#             onclick = row.get('onclick')
#             if onclick:
#                 # Extract the desired values from the onclick string
#                 parts = onclick.split("'")
#                 if len(parts) > 1:
#                     company_id = parts[1]
#                     company_name = parts[3]
#                     row_data.append(company_id)
#                     row_data.append(company_name)

#             data.append(row_data)

#         # Convert to DataFrame
#         columns = ['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company ID', 'Company Name']
#         df = pd.DataFrame(data, columns=columns)
        
#         df['Capital'] = df['Capital'].str.replace(',', '')
#         df['Capital'] = pd.to_numeric(df['Capital'])
#         df = df[(df['Capital'] >= 30000000)]
#         df = df.reset_index(drop=True)
        
#         df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company Name'])

#         # Concatenate df and result DataFrames
#         result = pd.concat([result, df], axis=0, ignore_index=True)
        
        
#     # Specify the file name and file path
#     file_name = value1+'.csv'
#     file_path = '/Users/bew/Downloads/WebScrapperFor_DataForThai/DataForThaiCompanyIdsByCategories/'

#     # Combine the file path and file name
#     full_path = os.path.join(file_path, file_name)

#     result.to_csv(full_path, sep=',', index=False, header=True, encoding='utf-8')
#     print(f"{value1} Complete")
#     df_null = []
#     result = pd.DataFrame(df_null, columns=columns)
#     result = result.drop(columns=['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company Name'])

#     # Reset the index of the final result DataFrame
#     result = result.reset_index(drop=True)

# result

In [8]:
print("finish")

finish


In [9]:
# value1 = '01111'

# file = '/Users/bew/Downloads/GetData_DataForThai/passthrough/'+value1+'_1.html'
# # Read the HTML file
# with open(file, 'r', encoding='utf-8') as file:
#     html_content = file.read()

# # Parse the HTML content
# soup = BeautifulSoup(html_content, 'lxml')

# # Find the table (modify as needed to target the correct table)
# table = soup.find('table', {'class': 'datatable'})

# # # Convert the table to a pandas DataFrame
# # df = pd.read_html(str(table))[0]

# # Initialize an empty list to store the rows
# data = []

# # Iterate over each row in the table
# for row in table.find_all('tr', class_='click-able-row'):
#     # Extract the text of each cell
#     cells = row.find_all('td')
#     row_data = [cell.text.strip() for cell in cells]

#     # Extract the onclick attribute
#     onclick = row.get('onclick')
#     if onclick:
#         # Extract the desired values from the onclick string
#         parts = onclick.split("'")
#         if len(parts) > 1:
#             company_id = parts[1]
#             company_name = parts[3]
#             row_data.append(company_id)
#             row_data.append(company_name)

#     data.append(row_data)

# # Convert to DataFrame
# columns = ['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company ID', 'Company Name']
# df = pd.DataFrame(data, columns=columns)
# # df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Hidden2', 'Province', 'District'])
# df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Hidden2', 'Province', 'District'])

# df

In [10]:
# value2 = '01112'

# file2 = '/Users/bew/Downloads/GetData_DataForThai/passthrough/'+value2+'_1.html'
# # Read the HTML file
# with open(file2, 'r', encoding='utf-8') as file:
#     html_content = file.read()

# # Parse the HTML content
# soup = BeautifulSoup(html_content, 'lxml')

# # Find the table (modify as needed to target the correct table)
# table = soup.find('table', {'class': 'datatable'})

# # # Convert the table to a pandas DataFrame
# # df = pd.read_html(str(table))[0]

# # Initialize an empty list to store the rows
# data = []

# # Iterate over each row in the table
# for row in table.find_all('tr', class_='click-able-row'):
#     # Extract the text of each cell
#     cells = row.find_all('td')
#     row_data = [cell.text.strip() for cell in cells]

#     # Extract the onclick attribute
#     onclick = row.get('onclick')
#     if onclick:
#         # Extract the desired values from the onclick string
#         parts = onclick.split("'")
#         if len(parts) > 1:
#             company_id = parts[1]
#             company_name = parts[3]
#             row_data.append(company_id)
#             row_data.append(company_name)

#     data.append(row_data)

# # Convert to DataFrame
# columns = ['Order', 'Name', 'Hidden1', 'Capital', 'Hidden2', 'Province', 'District', 'Company ID', 'Company Name']
# df2 = pd.DataFrame(data, columns=columns)
# # df = df.drop(columns=['Order', 'Name', 'Hidden1', 'Hidden2', 'Province', 'District'])
# df2 = df2.drop(columns=['Order', 'Name', 'Hidden1', 'Hidden2', 'Province', 'District'])

# df2

In [11]:
# file = '/Users/bew/Downloads/GetData_DataForThai/passthrough/01111_1.html'
# df = pd.read_html(file)
# df = df[1]
# # df.dtypes
# df

In [12]:
# i = '1'
# file = '/Users/bew/Downloads/GetData_DataForThai/passthrough/01112_1.html'
# df = pd.read_html(file)
# df = df[1]
# df

In [13]:
# # for index, row in df.iterrows():
# file_m = '/Users/bew/Downloads/GetData_DataForThai/passthrough/01112_1.html'
# df_m = pd.read_html(file_m)
# df_m = df_m[1]
# df_m

In [14]:
# result = pd.concat([df, df_m], axis=0)
# # print(result)

# result = result.reset_index(drop=True)
# result